# Trial with mapping
## important changes:
- use zookeper.py from my branch!!!
- __ getitem__ function changed
- forward changed
- check the path for '../artifacts' because I changed it

In [1]:
import optuna
from tqdm import tqdm
import logging
import sys
import torch
import os
from datetime import datetime
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import torch.utils.data
#from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split, Dataset, DataLoader
from PIL import Image
import zookeeper as zk  # convool_size & mappy
import pickle  # for artifact, maybe not useful

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

/home/margherita/src/ambiente/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


import shutil
    
data = pd.read_csv('../data/og/training_solutions_rev1.csv')
test = data.sample(frac=.1)
train = data.drop(test.index)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
train.to_csv('../data/training/training_solutions_rev1.csv', index_label=False)
test.to_csv('../data/test/test_solutions_rev1.csv', index_label=False)

for image in (train.GalaxyID.astype('string') + '.jpg').values:
    shutil.move(os.path.join('../data/og/', image), os.path.join('../data/training/', image))

for image in (test.GalaxyID.astype('string') + '.jpg').values:
    shutil.move(os.path.join('../data/og/', image), os.path.join('../data/test/', image))

train.head()


pd.read_csv('../data/test/test_solutions_rev1.csv').shape[0] + pd.read_csv('../data/training/training_solutions_rev1.csv').shape[0]


In [ ]:
class GalaxyJungle(Dataset):
    
    #the init function initializes the directory containing the image,
    #the annotations file,
    #and both transforms
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None, is_rgb=False):
        self.rgb = is_rgb
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    #returns number of samples in the dataset
    def __len__(self):
        return (self.img_labels).shape[0]

    #loads a sample from the dataset
     ############### MAPPED TARGET LABELS
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, str(self.img_labels.iloc[idx, 0])) + '.jpg'
        #retrieves the image
        image = Image.open(img_path)
        if not self.rgb: image = image.convert('L')
        #retrieves corresponding label
        label = self.img_labels.iloc[idx, 1:]
        #if possible, transform the image and the label into a tensor.
        if self.transform:
            image = self.transform(image)#.type(torch.float16)
        label=torch.tensor(label.to_numpy(), dtype=torch.float32, requires_grad=False) #cambiato in to_numpy perchè label.values restituisce un metodo e non un valore. In alternativa label.values() meno version proof
        label = zk.mappy(label)#.detach().clone()
        if self.target_transform:
            label = self.target_transform(label)
        return image, label, self.img_labels.iloc[idx, 0]
    

transfs = transforms.Compose([
    transforms.ToTensor(), # Riscala le immagini tra 0 e 1
    transforms.CenterCrop(324),
    transforms.Resize(128),
    transforms.RandomRotation(180)
    ])

DS = GalaxyJungle('../data/training/training_solutions_rev1.csv', '../data/training/', transfs)
training, test = random_split(DS, [.8, .2])
x=DS.__getitem__(0)
print(x[1].shape)

In [3]:
class GalaxyNet(nn.Module):
    def __init__(self, activation, initialization=False, is_rgb=False):
        super().__init__()
        
        rgb = 3 if is_rgb else 1
        input_size = 128
        num_labels = 37
        
        self.loss_dict = {'batch' : [], 'epoch' : [], 'vbatch' : [], 'vepoch' : []}
        self.activation = activation

        
        ## convolutional layers
        self.convs = nn.Sequential(
            nn.Conv2d(rgb, 16, 3, bias=False),
            nn.BatchNorm2d(16),
            self.activation(),
            

            nn.MaxPool2d(2),
            
            nn.Conv2d(16, 16, 3, bias=False),
            nn.BatchNorm2d(16),
            self.activation(),

            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, bias=False),
            nn.BatchNorm2d(32),
            self.activation(),
            

            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, bias=False),
            nn.BatchNorm2d(64),
            self.activation(),
            

            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, bias=False),
            nn.BatchNorm2d(128),
            self.activation(),
                               

            nn.MaxPool2d(2)
            )

        for layer in self.convs:
            if layer.__class__.__name__ == 'Conv2d': input_size = zk.convool_size(input_size, 3, 1)
            elif layer.__class__.__name__ == 'MaxPool2d': input_size = zk.convool_size(input_size, 2, 2)

        if input_size < 2: raise ValueError('You shrank too much dude.')
        print(f'Convs output size: {input_size}')

        input_linear = 128 * input_size * input_size
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_linear, 100),
            self.activation(),
            nn.Linear(100, num_labels)
            )
        
        if initialization: self.init_weights()



    def forward(self, x):
        x = self.convs(x)
        x = self.fc(x)
        x= zk.mappy2D(x)
        #print('After mappy:', x.shape)
        return x


    def init_weights(self):
        if self.activation == nn.ReLU:
            nonlin = 'relu'
            a = 0
        elif self.activation == nn.LeakyReLU:
            nonlin = 'leaky_relu'
            a = .01
        
        # Init convolutional parameters
        for layer in self.convs: 
            if layer.__class__.__name__ == 'Conv2d': nn.init.kaiming_normal_(layer.weight, a=a, nonlinearity=nonlin)
        

        # Init linear parameters
        for i in (1, -1): nn.init.constant_(self.fc[i].bias, 0)
        nn.init.kaiming_normal_(self.fc[1].weight, a=a, nonlinearity=nonlin)
        nn.init.xavier_uniform_(self.fc[-1].weight)      
        


    def log_the_loss(self, item,epoch=False): # per avere una history della loss???
        verbose=False
        train = self.__getstate__()['training']
        if verbose: print(train)
        if epoch and train:
            self.loss_dict['epoch'].append(item) ### get state of the model so you can ditch the validation parameter
        elif not epoch and train:
            self.loss_dict['batch'].append(item)
        elif not train and epoch:
            self.loss_dict['vepoch'].append(item)
        elif not train and not epoch:
            self.loss_dict['vbatch'].append(item)
        return item


In [6]:
g=GalaxyNet(activation=nn.ReLU, initialization=True, is_rgb=False)
print(g)

Convs output size: 2
GalaxyNet(
  (convs): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [4]:
def one_epoch_train(model, train_loader, optimizer, loss_function, verbose=False):
    running_loss = 0
    model.train()
    for i, data in tqdm(enumerate(train_loader)):
        inputs,labels, _ = data
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss=loss_function(outputs, labels)
        loss.backward()
        optimizer.step() # fa update del parameter
        RMSEloss = np.sqrt(loss.item())
        running_loss += RMSEloss
        if verbose and i%10 ==0: print(f'Batch {i+1}/{len(train_loader)} - Loss: {RMSEloss:.3f}')

        model.log_the_loss(RMSEloss, epoch=False)
    epochmean_loss = running_loss / len(train_loader)
    print(f'\nLoss: {epochmean_loss}')
    model.log_the_loss(epochmean_loss, epoch=True)
    last_loss = RMSEloss
    print(f"Last loss: {last_loss}")
    return epochmean_loss



def one_epoch_eval(model, test_loader, loss_function, verbose=False):
    model.eval()
    running_validation_loss = 0.
   
    with torch.no_grad(): # deactivates gradient evaluation
        for i, vdata in tqdm(enumerate(test_loader)):
            inputs,labels, _ = vdata
            inputs,labels= inputs.to(device), labels.to(device)
            outputs = model(inputs)#, activation=F.relu)
            loss = loss_function(outputs ,labels)
            RMSEloss = np.sqrt(loss.item())
            running_validation_loss += RMSEloss
            model.log_the_loss(RMSEloss,epoch=False)
    mean_vloss=model.log_the_loss(running_validation_loss / len(test_loader),epoch=True)
    print(f"Validation Loss: {mean_vloss}\n---")
    return mean_vloss

In [5]:
DS = GalaxyJungle('../data/training/training_solutions_rev1.csv', '../data/training/', transfs)
training, test = random_split(DS, [.8, .2])

# MARGH: ARTIFACTS È FUORI DALLA CARTELLA. CONTROLLARE IL PATH
artifact_store = optuna.artifacts.FileSystemArtifactStore(base_path='../artifacts')

def objective(trial:optuna.Trial):
    epochs = 50
    loss_function = nn.MSELoss()
    train_loader = DataLoader(training, batch_size=32, shuffle=True, num_workers=os.cpu_count())
    test_loader = DataLoader(test, batch_size=32, shuffle=False, num_workers=os.cpu_count())    
    
    # Trial choices
    activation = trial.suggest_categorical("activation", ['ReLU', 'LeakyReLU'])
    optimizer = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'AdamW', 'RMSprop', 'Adagrad', 'NAdam']) #AdamW è suggerito per CNN.
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True) #log true cerca i valori in scala logaritmica
    momentum = trial.suggest_float("momentum", 0.5, 0.9, step=0.1) #per SGD
    initialization = trial.suggest_categorical('init weight', [True, False])
    
    # Training phase
    activation = getattr(nn, activation)
    model = GalaxyNet(activation, initialization).to(device)
    if optimizer in ('SGD', "RMSprop"): optimizer = getattr(optim, optimizer)(model.parameters(), lr=learning_rate, momentum = momentum)
    else: optimizer = getattr(optim, optimizer)(model.parameters(), lr=learning_rate)
    
    
    for epoch in range(epochs):
        print(f'Training epoch {epoch}')
        one_epoch_train(model, train_loader, optimizer, loss_function, verbose=False)

        print(f'Validation epoch {epoch}')
        epoch_last_val_loss = one_epoch_eval(model, test_loader, loss_function, verbose=False)
        trial.report(epoch_last_val_loss, epoch)


        if trial.should_prune(): raise optuna.TrialPruned()

    with open('model.pickle', 'wb') as fout: pickle.dump(model, fout)
    art_id = optuna.artifacts.upload_artifact(artifact_store=artifact_store, file_path='model.pickle', study_or_trial=trial.study)
    trial.set_user_attr('artifact_id', art_id)

    
    return epoch_last_val_loss

# MODEL LOAD + NEW TRAINING

In [6]:
model = pd.read_pickle('../artifacts/4ec97088-5fca-463c-a02b-a8d92f855242')
print(model)

GalaxyNet(
  (convs): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (1

In [11]:
model = model
learning_rate = 0.0008509005973164451
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
epochs = 50
loss_function = nn.MSELoss()
train_loader = DataLoader(training, batch_size=32, shuffle=True, num_workers=os.cpu_count())
test_loader = DataLoader(test, batch_size=32, shuffle=False, num_workers=os.cpu_count())
for epoch in range(epochs):
    print(f'Training epoch {epoch}')
    one_epoch_train(model, train_loader, optimizer, loss_function, verbose=False)
    print(f'Validation epoch {epoch}')
    epoch_last_val_loss = one_epoch_eval(model, test_loader, loss_function, verbose=False)


Training epoch 0


1386it [00:32, 43.05it/s]


Loss: 0.06345913402083213
Last loss: 0.058063199191764885
Validation epoch 0



347it [00:05, 62.86it/s]

Validation Loss: 0.06448081397251235
---
Training epoch 1


...

Validation Loss: 0.06376190268938284
---
Training epoch 49



1386it [00:18, 75.53it/s]


Loss: 0.061622734567953016
Last loss: 0.06664905616626042
Validation epoch 49



347it [00:03, 106.98it/s]

Validation Loss: 0.06257973982103282
---


la loss è del training, validation è quella della validation
Validation quando sale non ci piace più.

In [13]:
model.loss_dict.keys()

len(model.loss_dict['epoch'])


50

In [31]:
torch.set_printoptions(sci_mode=False)

# LABELING THE UNLABELED IMAGES

model.eval()
   
with torch.no_grad(): # deactivates gradient evaluation
    for i, vdata in tqdm(enumerate(test_loader)):
        inputs,labels, _ = vdata
        inputs,labels= inputs.to(device), labels.to(device)
        outputs = model(inputs)#, activation=F.relu)



img_path = os.path.join(self.img_dir, str(self.img_labels.iloc[idx, 0])) + '.jpg' #retrieves the imageù
img_path= '../data/test_old/100232.jpg'
image = Image.open(img_path).convert('L')

transform= transforms.Compose([
    transforms.ToTensor(), # Riscala le immagini tra 0 e 1
    transforms.CenterCrop(324),
    transforms.Resize(128),
])
image = transform(image)  # Add batch dimension and move to device

print(image.shape)


img_path = os.path.join('../test_old', str(img_labels.iloc[idx, 0])) + '.jpg'
image = Image.open(img_path).convert('L')

transform= transforms.Compose([
    transforms.ToTensor(), # Riscala le immagini tra 0 e 1
    transforms.CenterCrop(324),
    transforms.Resize(128),
])
image = transform(image)  # Add batch dimension and move to device

print(image.shape)
model.eval()
with torch.no_grad(): # deactivates gradient evaluation
    image=image.unsqueeze(0).to(device)
    outputs=model(image)

print(outputs)

labels= ['E0','E3','E6','S0a_eon','SB0a_eon','Scd_eon','SoB','SoA','SAa','SAb','SAc','SAd','SBa','SBb','SBc','SBd','A']
df_out=pd.DataFrame(outputs.cpu().numpy(), columns=labels)
print(df_out)

In [86]:

outs=[]
names=[]
transform= transforms.Compose([
    transforms.ToTensor(), # Riscala le immagini tra 0 e 1
    transforms.CenterCrop(324),
    transforms.Resize(128),
])

j=0
model.eval()
# 'GalaxyID',
labels= ['E0','E3','E6','S0a_eon','SB0a_eon','Scd_eon','SoB','SoA','SAa','SAb','SAc','SAd','SBa','SBb','SBc','SBd','A']
for i in os.listdir('../data/test/'):
    if i.endswith('.jpg'):
        j+=1
        img_path = os.path.join('../data/test/', i)
        image = Image.open(img_path).convert('L')
        #plt.imshow(image, cmap='gray')       
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(image)
            outs.append(outputs.cpu().numpy()[0])
        names.append(i)
        #if j==20: break

df_out = pd.DataFrame(outs, columns=labels)

#set negative values to zero
df_out[df_out<0]=0

#to get the final label
finallbs=[]
for i in range(len(df_out)):
    finallbs.append(df_out.columns[np.argmax(df_out.iloc[i,:])])
#print(finallbs)
df_out['final_labels']=finallbs

#name column and sort!
df_out. insert(0, 'Name', names)
df_out.sort_values('Name',inplace=True)

display(df_out)


,Name,E0,E3,E6,S0a_eon,SB0a_eon,Scd_eon,SoB,SoA,SAa,SAb,SAc,SAd,SBa,SBb,SBc,SBd,A,final_labels
1729,100078.jpg,0.464655,0.091942,0.000856,0.012567,0.000000,0.000217,0.028111,0.279394,0.012123,0.044611,0.018703,0.001055,0.001220,0.004488,0.001882,0.000106,0.025332,E0
5280,100128.jpg,0.636307,0.127252,0.000000,0.003772,0.000310,0.000000,0.012888,0.148794,0.005105,0.012472,0.006830,0.001878,0.000442,0.001080,0.000592,0.000163,0.038725,E0
4679,100143.jpg,0.000000,0.007974,0.118648,0.572754,0.144260,0.104311,0.013689,0.021367,0.002967,0.005889,0.005138,0.002547,0.001901,0.003773,0.003292,0.001632,0.005633,S0a_eon
128,100335.jpg,0.001162,0.082569,0.118097,0.023997,0.014627,0.117828,0.180499,0.126636,0.008122,0.007879,0.047847,0.064966,0.011577,0.011230,0.068199,0.092599,0.029378,SoB
153,100367.jpg,0.089447,0.490372,0.002527,0.017965,0.002555,0.006938,0.113717,0.160466,0.009653,0.029078,0.015172,0.002845,0.006841,0.020607,0.010752,0.002016,0.021128,E3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5849,998723.jpg,0.012276,0.264975,0.086866,0.078834,0.005024,0.024225,0.071169,0.237244,0.009846,0.069166,0.081092,0.007852,0.002954,0.020748,0.024326,0.002356,0.005489,E3
3377,998735.jpg,0.092654,0.196791,0.000000,0.023573,0.001071,0.003362,0.172584,0.161216,0.017255,0.090593,0.055645,0.002661,0.018471,0.096981,0.059568,0.002848,0.009410,E3
2708,999137.jpg,0.005973,0.000000,0.000000,0.027779,0.000000,0.006337,0.012745,0.008344,0.022701,0.160434,0.189193,0.019843,0.034674,0.245050,0.288976,0.030309,0.000000,SBc
5183,999569.jpg,0.082922,0.652412,0.000016,0.007581,0.000000,0.007945,0.016677,0.168443,0.009597,0.025383,0.011737,0.002764,0.000950,0.002513,0.001162,0.000274,0.021669,E3


df.sort_values('Name',inplace=True)
df

df=df.drop(columns='Name')
finallbs=[]
for i in range(len(df)):
    finallbs.append(df.columns[np.argmax(df.iloc[i,:])])
print(finallbs)


df['final_labels']=finallbs
df.insert(0,'Name',names)

df.sort_values('Name', inplace=True)

In [87]:
df_out

,Name,E0,E3,E6,S0a_eon,SB0a_eon,Scd_eon,SoB,SoA,SAa,SAb,SAc,SAd,SBa,SBb,SBc,SBd,A,final_labels
1729,100078.jpg,0.464655,0.091942,0.000856,0.012567,0.000000,0.000217,0.028111,0.279394,0.012123,0.044611,0.018703,0.001055,0.001220,0.004488,0.001882,0.000106,0.025332,E0
5280,100128.jpg,0.636307,0.127252,0.000000,0.003772,0.000310,0.000000,0.012888,0.148794,0.005105,0.012472,0.006830,0.001878,0.000442,0.001080,0.000592,0.000163,0.038725,E0
4679,100143.jpg,0.000000,0.007974,0.118648,0.572754,0.144260,0.104311,0.013689,0.021367,0.002967,0.005889,0.005138,0.002547,0.001901,0.003773,0.003292,0.001632,0.005633,S0a_eon
128,100335.jpg,0.001162,0.082569,0.118097,0.023997,0.014627,0.117828,0.180499,0.126636,0.008122,0.007879,0.047847,0.064966,0.011577,0.011230,0.068199,0.092599,0.029378,SoB
153,100367.jpg,0.089447,0.490372,0.002527,0.017965,0.002555,0.006938,0.113717,0.160466,0.009653,0.029078,0.015172,0.002845,0.006841,0.020607,0.010752,0.002016,0.021128,E3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5849,998723.jpg,0.012276,0.264975,0.086866,0.078834,0.005024,0.024225,0.071169,0.237244,0.009846,0.069166,0.081092,0.007852,0.002954,0.020748,0.024326,0.002356,0.005489,E3
3377,998735.jpg,0.092654,0.196791,0.000000,0.023573,0.001071,0.003362,0.172584,0.161216,0.017255,0.090593,0.055645,0.002661,0.018471,0.096981,0.059568,0.002848,0.009410,E3
2708,999137.jpg,0.005973,0.000000,0.000000,0.027779,0.000000,0.006337,0.012745,0.008344,0.022701,0.160434,0.189193,0.019843,0.034674,0.245050,0.288976,0.030309,0.000000,SBc
5183,999569.jpg,0.082922,0.652412,0.000016,0.007581,0.000000,0.007945,0.016677,0.168443,0.009597,0.025383,0.011737,0.002764,0.000950,0.002513,0.001162,0.000274,0.021669,E3


In [88]:
df_out.to_csv('../data/test_old/galaxyjungle_test_gio.csv', index=False)

# OPTUNA DASHBOARD


In [ ]:
#optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "InverseJagZooNet"
storage_name = "sqlite:///{}.db".format(study_name)

from optuna_dashboard import run_server
run_server(storage_name)

In [ ]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "JAGZooNet"
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective, n_trials=1)